In [1]:
import cv2
from ultralytics import YOLO
from datetime import datetime
import time

import pandas as pd
columns = ['preReadTime', 'postReadTime']
metaDf = pd.DataFrame(columns=columns)

model = YOLO("yolo11n-pose.pt")

# Export the model to NCNN format
model.export(format="ncnn")  # creates 'yolo11n_ncnn_model'

# Load the exported NCNN model
ncnn_model = YOLO("yolo11n-pose.torchscript")


# Initialize the webcam (0 is the default camera)
timeBeforeCapDefined = datetime.now() 
cap = cv2.VideoCapture(0)
timeAfterCapDefined = datetime.now() 

cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()


initalFrameReadStart = datetime.now()
ret, frame = cap.read()
initalFrameReadEnd = datetime.now()


Ultralytics 8.3.18 🚀 Python-3.11.10 torch-2.5.0+cu124 CPU (AMD Ryzen 5 3600 6-Core Processor)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs

PyTorch: starting from 'yolo11n-pose.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 56, 8400) (6.0 MB)

TorchScript: starting export with torch 2.5.0+cu124...
TorchScript: export success ✅ 1.9s, saved as 'yolo11n-pose.torchscript' (11.6 MB)

NCNN: starting export with NCNN 1.0.20240820...
NCNN: running '/home/chowder/anaconda3/envs/vision/lib/python3.11/site-packages/ultralytics/pnnx yolo11n-pose.torchscript ncnnparam=yolo11n-pose_ncnn_model/model.ncnn.param ncnnbin=yolo11n-pose_ncnn_model/model.ncnn.bin ncnnpy=yolo11n-pose_ncnn_model/model_ncnn.py pnnxparam=yolo11n-pose_ncnn_model/model.pnnx.param pnnxbin=yolo11n-pose_ncnn_model/model.pnnx.bin pnnxpy=yolo11n-pose_ncnn_model/model_pnnx.py pnnxonnx=yolo11n-pose_ncnn_model/model.pnnx.onnx fp16=0 device=cpu inputshape="[1, 3, 640, 640]"'


pnnxparam = yolo11n-pose_ncnn_model/model.pnnx.param
pnnxbin = yolo11n-pose_ncnn_model/model.pnnx.bin
pnnxpy = yolo11n-pose_ncnn_model/model_pnnx.py
pnnxonnx = yolo11n-pose_ncnn_model/model.pnnx.onnx
ncnnparam = yolo11n-pose_ncnn_model/model.ncnn.param
ncnnbin = yolo11n-pose_ncnn_model/model.ncnn.bin
ncnnpy = yolo11n-pose_ncnn_model/model_ncnn.py
fp16 = 0
optlevel = 2
device = cpu
inputshape = [1,3,640,640]f32
inputshape2 = 
customop = 
moduleop = 
############# pass_level0
inline module = torch.nn.modules.linear.Identity
inline module = ultralytics.nn.modules.block.Attention
inline module = ultralytics.nn.modules.block.Bottleneck
inline module = ultralytics.nn.modules.block.C2PSA
inline module = ultralytics.nn.modules.block.C3k
inline module = ultralytics.nn.modules.block.C3k2
inline module = ultralytics.nn.modules.block.DFL
inline module = ultralytics.nn.modules.block.PSABlock
inline module = ultralytics.nn.modules.block.SPPF
inline module = ultralytics.nn.modules.conv.Concat
inline 

NCNN: export success ✅ 2.0s, saved as 'yolo11n-pose_ncnn_model' (11.2 MB)

Export complete (4.2s)
Results saved to /home/chowder/Documents/videoProcessing/homeVideo
Predict:         yolo predict task=pose model=yolo11n-pose_ncnn_model imgsz=640  
Validate:        yolo val task=pose model=yolo11n-pose_ncnn_model imgsz=640 data=/ultralytics/ultralytics/cfg/datasets/coco-pose.yaml  
Visualize:       https://netron.app


############# pass_ncnn


In [2]:
timeAfterCapDefined - timeBeforeCapDefined

datetime.timedelta(microseconds=268074)

In [3]:
initalFrameReadEnd - initalFrameReadStart

datetime.timedelta(seconds=1, microseconds=54519)

In [4]:
# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'avc1')
output = cv2.VideoWriter('output.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

In [16]:
def writeFrames(output, frameList):
    for frame in frameList:
        output.write(frame)
    return True

def writeFramesWrapper(args):
    return writeFrames(*args)

def runYolo(frame):
    # return True
    print('running yolo')
    result = ncnn_model(frame)
    print('done running yolo')
    return len(result[0].names) > 0

def runYoloWrapper(args):
    return runYolo(*args)

In [17]:
runYolo(frame)

running yolo

0: 640x640 1 person, 76.4ms
Speed: 4.3ms preprocess, 76.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
done running yolo


True

In [18]:
from multiprocessing import Pool
with Pool() as pool:
    preModelInferenceTime = datetime.now()
    print('about to send for processing')
    lastFramePeople_result = pool.apply_async(runYolo, (frame,))
    print(f"sent for processing! in {datetime.now() - preModelInferenceTime}")
    print(lastFramePeople_result.ready())
    time.sleep(1)
    print(lastFramePeople_result.ready())
    lfp = lastFramePeople_result.get(timeout=3) #this shouldn't take close to 15 seconds
    postModelInferenceTime = datetime.now()
    print(f"the model took {postModelInferenceTime - preModelInferenceTime} to run")

running yolo

about to send for processing
sent for processing! in 0:00:00.000450
False
False


KeyboardInterrupt: 

In [14]:
from multiprocessing import Pool

import time
frameCount = 0
mybuffer = []
people = False
activeSections = 0


with Pool() as pool:

    preModelInferenceTime = datetime.now()
    print('about to send for processing')
    lastFramePeople_result = pool.map_async(runYolo, [frame])
    print("sent for processing!")
    lfp = lastFramePeople_result.get() #this shouldn't take close to 15 seconds
    postModelInferenceTime = datetime.now()
    print(f"the model took {postModelInferenceTime - preModelInferenceTime} to run")


    while frameCount < 2000:
        # delay till 100ms
        msToDelay = 100 - ((datetime.now().microsecond / 1000) % 100)
        time.sleep(msToDelay/1000)
        
        #logging and frame cap
        rowToAdd = {}
        lastTimestamp = datetime.now()
        rowToAdd['preReadTime'] = lastTimestamp
        ret, frame = cap.read()
        rowToAdd['postReadTime'] = datetime.now()
        metaDf.loc[len(metaDf)] = rowToAdd
        mybuffer.append(frame)

        if datetime.now().second % 15 == 0 and datetime.now().microsecond < 100_000:
            # check the result of the last processed frame
            if not lastFramePeople_result.ready(): print("waiting for processing")
            lfp = lastFramePeople_result.get() #this shouldn't take close to 15 seconds

            # if there was people then consider the last and next 15 seconds as active
            if lfp:
                activeSections = 2
                print('saw someone')
            
            
            if not write_result.ready(): print("waiting for processing")
            write_result.get() #this shouldn't take close to 15 seconds
            
            if activeSections > 0:
                #async write all frames up untill this point
                write_result = pool.map_async(writeFramesWrapper, [(output, mybuffer)])
                activeSections -= 1
            else:
                # just write mybuffer[-1]
                write_result = pool.map_async(writeFramesWrapper, [(output,[mybuffer[-1]])])


            # send the most recent frame off for processing
            lastFramePeople_result = pool.map_async(runYoloWrapper, [(ncnn_model, mybuffer[-1])])

            mybuffer = []


        frameCount += 1



about to send for processing
sent for processing!


KeyboardInterrupt: 

In [ ]:
# Release the webcam and close the output file
cap.release()
output.release()

In [5]:
metaDf['timeBetweenReads'] =  metaDf['preReadTime'].shift(-1) - metaDf['preReadTime']
(metaDf['timeBetweenReads'].dt.total_seconds() * 1000).describe()

count    199.000000
mean     100.000040
std        0.353543
min       96.995000
25%       99.996000
50%      100.000000
75%      100.005000
max      103.005000
Name: timeBetweenReads, dtype: float64

In [6]:
for e in metaDf['preReadTime'].tolist():
    print(e)

2024-11-01 08:59:07.700074
2024-11-01 08:59:07.800073
2024-11-01 08:59:07.900072
2024-11-01 08:59:08.000073
2024-11-01 08:59:08.100188
2024-11-01 08:59:08.200069
2024-11-01 08:59:08.300065
2024-11-01 08:59:08.400065
2024-11-01 08:59:08.500073
2024-11-01 08:59:08.600184
2024-11-01 08:59:08.700077
2024-11-01 08:59:08.800075
2024-11-01 08:59:08.900122
2024-11-01 08:59:09.000077
2024-11-01 08:59:09.100074
2024-11-01 08:59:09.200074
2024-11-01 08:59:09.300074
2024-11-01 08:59:09.400070
2024-11-01 08:59:09.500074
2024-11-01 08:59:09.600064
2024-11-01 08:59:09.700073
2024-11-01 08:59:09.800077
2024-11-01 08:59:09.900079
2024-11-01 08:59:10.000077
2024-11-01 08:59:10.100073
2024-11-01 08:59:10.200081
2024-11-01 08:59:10.300073
2024-11-01 08:59:10.400076
2024-11-01 08:59:10.500075
2024-11-01 08:59:10.600072
2024-11-01 08:59:10.700074
2024-11-01 08:59:10.800077
2024-11-01 08:59:10.900074
2024-11-01 08:59:11.000081
2024-11-01 08:59:11.100078
2024-11-01 08:59:11.200077
2024-11-01 08:59:11.300076
2